In [1]:
import pandas as pd
import csv
import os

'''Features'''
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import label_binarize

df = pd.read_csv (r'C:\Users\Steven\Desktop\data folder\Interview_Mapping.csv', encoding='unicode_escape')
df.head()
#df.head(insert number) will show all the stuff




,Judgements,Area.of.Law,Content
0,LNIND_1951_CAL_115,Criminal Laws, <KYDishonestly receiving stolen property Par...
1,LNIND_1954_CAL_37,Property Laws,Parties\nSatya Kinkar Dutt Versus Kiron Chandr...
2,LNIND_1954_CAL_72,Criminal Procedure,Parties\nBholaram Dalmia Versus State\nHigh Co...
3,LNIND_1953_CAL_151,Limitation,Parties\nSri Madan Mohon Jew Versus Bejoyabati...
4,LNIND_1980_CAL_65,Company Law,\n\nParties\nIndian Bank Versus Bengal Potteri...


In [2]:
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
import itertools, string, operator, re, unicodedata, nltk
from operator import itemgetter
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer, RegexpTokenizer
from bs4 import BeautifulSoup
import numpy as np
from itertools import combinations
from gensim.models import Phrases
from collections import Counter

#Contraction map
c_dict = {
  "ain't": "am not",
  "aren't": "are not",
  "can't": "cannot",
  "can't've": "cannot have",
  "'cause": "because",
  "could've": "could have",
  "couldn't": "could not",
  "couldn't've": "could not have",
  "didn't": "did not",
  "doesn't": "does not",
  "don't": "do not",
  "hadn't": "had not",
  "hadn't've": "had not have",
  "hasn't": "has not",
  "haven't": "have not",
  "he'd": "he would",
  "he'd've": "he would have",
  "he'll": "he will",
  "he'll've": "he will have",
  "he's": "he is",
  "how'd": "how did",
  "how'd'y": "how do you",
  "how'll": "how will",
  "how's": "how is",
  "i'd": "I would",
  "i'd've": "I would have",
  "i'll": "I will",
  "i'll've": "I will have",
  "i'm": "I am",
  "i've": "I have",
  "isn't": "is not",
  "it'd": "it had",
  "it'd've": "it would have",
  "it'll": "it will",
  "it'll've": "it will have",
  "it's": "it is",
  "let's": "let us",
  "ma'am": "madam",
  "mayn't": "may not",
  "might've": "might have",
  "mightn't": "might not",
  "mightn't've": "might not have",
  "must've": "must have",
  "mustn't": "must not",
  "mustn't've": "must not have",
  "needn't": "need not",
  "needn't've": "need not have",
  "o'clock": "of the clock",
  "oughtn't": "ought not",
  "oughtn't've": "ought not have",
  "shan't": "shall not",
  "sha'n't": "shall not",
  "shan't've": "shall not have",
  "she'd": "she would",
  "she'd've": "she would have",
  "she'll": "she will",
  "she'll've": "she will have",
  "she's": "she is",
  "should've": "should have",
  "shouldn't": "should not",
  "shouldn't've": "should not have",
  "so've": "so have",
  "so's": "so is",
  "that'd": "that would",
  "that'd've": "that would have",
  "that's": "that is",
  "there'd": "there had",
  "there'd've": "there would have",
  "there's": "there is",
  "they'd": "they would",
  "they'd've": "they would have",
  "they'll": "they will",
  "they'll've": "they will have",
  "they're": "they are",
  "they've": "they have",
  "to've": "to have",
  "wasn't": "was not",
  "we'd": "we had",
  "we'd've": "we would have",
  "we'll": "we will",
  "we'll've": "we will have",
  "we're": "we are",
  "we've": "we have",
  "weren't": "were not",
  "what'll": "what will",
  "what'll've": "what will have",
  "what're": "what are",
  "what's": "what is",
  "what've": "what have",
  "when's": "when is",
  "when've": "when have",
  "where'd": "where did",
  "where's": "where is",
  "where've": "where have",
  "who'll": "who will",
  "who'll've": "who will have",
  "who's": "who is",
  "who've": "who have",
  "why's": "why is",
  "why've": "why have",
  "will've": "will have",
  "won't": "will not",
  "won't've": "will not have",
  "would've": "would have",
  "wouldn't": "would not",
  "wouldn't've": "would not have",
  "y'all": "you all",
  "y'alls": "you alls",
  "y'all'd": "you all would",
  "y'all'd've": "you all would have",
  "y'all're": "you all are",
  "y'all've": "you all have",
  "you'd": "you had",
  "you'd've": "you would have",
  "you'll": "you you will",
  "you'll've": "you you will have",
  "you're": "you are",
  "you've": "you have"
}

c_re = re.compile('(%s)' % '|'.join(c_dict.keys()))

add_stop = ['', ' ', 'say', 's', 'u', 'ap', 'afp', '...', 'n', '\\']

stop_words = ENGLISH_STOP_WORDS.union(add_stop)

tokenizer = TweetTokenizer()
pattern = r"(?u)\b\w\w+\b" 

lemmatizer = WordNetLemmatizer()

punc = list(set(string.punctuation))

def casual_tokenizer(text): #Splits words on white spaces (leaves contractions intact) and splits out trailing punctuation
    tokens = tokenizer.tokenize(text)
    return tokens

#Function to replace the nltk pos tags with the corresponding wordnet pos tag to use the wordnet lemmatizer
def get_word_net_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None
    
def lemma_wordnet(tagged_text):
    final = []
    for word, tag in tagged_text:
        wordnet_tag = get_word_net_pos(tag)
        if wordnet_tag is None:
            final.append(lemmatizer.lemmatize(word))
        else:
            final.append(lemmatizer.lemmatize(word, pos=wordnet_tag))
    return final

def expandContractions(text, c_re=c_re):
    def replace(match):
        return c_dict[match.group(0)]
    return c_re.sub(replace, text)

def process_text(text):
    soup = BeautifulSoup(text, "lxml")
    tags_del = soup.get_text()
    no_html = re.sub('<[^>]*>', '', tags_del)
    tokenized = casual_tokenizer(no_html)
    lower = [item.lower() for item in tokenized]
    decontract = [expandContractions(item, c_re=c_re) for item in lower]
    tagged = nltk.pos_tag(decontract)
    lemma = lemma_wordnet(tagged)
    no_num = [re.sub('[0-9]+', '', each) for each in lemma]
    no_punc = [w for w in no_num if w not in punc]
    no_stop = [w for w in no_punc if w not in stop_words]
    return no_stop

def word_count(text):
    return len(str(text).split(' '))

def word_freq(clean_text_list, top_n):
    """
    Word Frequency
    """
    flat = [item for sublist in clean_text_list for item in sublist]
    with_counts = Counter(flat)
    top = with_counts.most_common(top_n)
    word = [each[0] for each in top]
    num = [each[1] for each in top]
    return pd.DataFrame([word, num]).T



In [3]:

#Apply the function to preprocess the text. Tokenize, lower, expand contactions, lemmatize, remove punctuation, numbers and stop words

df['clean_text'] = df['Content'].apply(process_text)
df.head(80)

,Judgements,Area.of.Law,Content,clean_text
0,LNIND_1951_CAL_115,Criminal Laws, <KYDishonestly receiving stolen property Par...,"[, bharadwaj, singh, versus, state, west, ben..."
1,LNIND_1954_CAL_37,Property Laws,Parties\nSatya Kinkar Dutt Versus Kiron Chandr...,"[party, satya, kinkar, dutt, versus, kiron, ch..."
2,LNIND_1954_CAL_72,Criminal Procedure,Parties\nBholaram Dalmia Versus State\nHigh Co...,"[party, bholaram, dalmia, versus, state, high,..."
3,LNIND_1953_CAL_151,Limitation,Parties\nSri Madan Mohon Jew Versus Bejoyabati...,"[party, sri, madan, mohon, jew, versus, bejoya..."
4,LNIND_1980_CAL_65,Company Law,\n\nParties\nIndian Bank Versus Bengal Potteri...,"[party, indian, bank, versus, bengal, pottery,..."
5,LNIND_1951_CAL_212,Civil Procedure,\n\nParties\nGurudasi Devi Versus Chandi Chara...,"[party, gurudasi, devi, versus, chandi, charan..."
6,LNIND_1952_CAL_129,Succession Laws, Parties\nSurja Kumar Sardar Versus Manmatha...,"[, party, surja, kumar, sardar, versus, manma..."
7,LNIND_1976_CAL_157,Succession Laws,\n\nParties\nSatyanarayan Saraf Versus Assista...,"[party, satyanarayan, saraf, versus, assistant..."
8,LNIND_1976_CAL_57,Civil Procedure,<LG> Code of Civil Procedure  </LG><PV>Sectio...,"[code, civil, procedure, , section, , order,..."
9,LNIND_1957_CAL_37,Local Government,\n\nParties\nCorporation of Calcutta Versus Un...,"[party, corporation, calcutta, versus, union, ..."


In [4]:

#Top 20 most frequent words for all the articles
cl_text_list = df['clean_text'].tolist()
wf = word_freq(cl_text_list, 20)
wf.head(20)

,0,1
0,court,4999
1,act,4954
2,case,4678
3,section,4402
4,order,3859
5,make,3434
6,petitioner,2690
7,mr,2319
8,party,2273
9,appeal,1977


In [5]:
#Avg word count by category
df['word_count'] = df['Content'].apply(word_count)
avg_wc = df.groupby('Area.of.Law').mean().reset_index()
avg_wc[['Area.of.Law','word_count']]

,Area.of.Law,word_count
0,Administrative Law,2273.666667
1,Alternative Dispute Resolution,2804.666667
2,Arbitration,2681.166667
3,Banking And Finance,2598.500000
4,Civil Laws,1673.777778
5,Civil Procedure,1821.400000
6,Company Law,2387.411765
7,Constitution,2399.923077
8,Consumer Law,1264.000000
9,Contempt Of Court,1518.000000


In [85]:
#Just testing stuff
#Preparing the dataframes

#Splitting the df into the different categories
#df_Administrative_Law = df.loc[df['Area.of.Law'] == 'Administrative Law'] 
#df_Alternative_Dispute_Resolution = df.loc[df['Area.of.Law'] =='Alternative Dispute Resolution']
#df_Arbitration = df.loc[df['Area.of.Law'] == 'Arbitration'] 
#df_Banking_And_Finance = df.loc[df['Area.of.Law'] =='Banking And Finance']
#df_Civil_Laws = df.loc[df['Area.of.Law'] =='Civil Laws']
#df_Civil_Procedure = df.loc[df['Area.of.Law'] =='Civil Procedure']
#df_Company_Law = df.loc[df['Area.of.Law'] =='Company Law']
#df_Constitution = df.loc[df['Area.of.Law'] =='Constitution']
#df_Criminal_Laws = df.loc[df['Area.of.Law'] =='Criminal Laws']
#df_Criminal_Procedure = df.loc[df['Area.of.Law'] =='Criminal Procedure']
#df_Customs = df.loc[df['Area.of.Law'] =='Customs']
#df_Employment_And_Labour_Law = df.loc[df['Area.of.Law'] =='Employment And Labour Law']
#df_Evidence = df.loc[df['Area.of.Law'] =='Evidence']
#df_Family_Law = df.loc[df['Area.of.Law'] =='Family Law']
#df_Government_Contracts = df.loc[df['Area.of.Law'] =='Government Contracts']
#df_Income_Tax= df.loc[df['Area.of.Law'] =='Income Tax']
#df_Intellectual_Property_Laws = df.loc[df['Area.of.Law'] =='Intellectual Property Laws']
#df_Limitation = df.loc[df['Area.of.Law'] =='Limitation']
#df_Local_Government   = df.loc[df['Area.of.Law'] =='Local Government']
#df_Property_Laws = df.loc[df['Area.of.Law'] =='Property Laws']
#df_Registration = df.loc[df['Area.of.Law'] =='Registration']
#df_Sales_Tax_And_Vat = df.loc[df['Area.of.Law'] =='Sales Tax And Vat']
#df_Service_Law = df.loc[df['Area.of.Law'] =='Service Law']
#df_Succession_Laws   = df.loc[df['Area.of.Law'] =='Succession Laws']
#df_Tenancy_Laws = df.loc[df['Area.of.Law'] =='Tenancy Laws']

#Appending the dfs back together
#LE = LabelEncoder()

                   
#df['content_num'] = LE.fit_transform(df['Content'])

#display(df.groupby(['Area.of.Law'])['Content'].count())
#display(df['Area.of.Law'].unique())
#display(df['content_num'].unique())

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import tree


texts = df['clean_text'].astype('str')

tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2), 
                                   min_df = 2, 
                                   max_df = .95)

X = tfidf_vectorizer.fit_transform(texts) #features
y = df['Area.of.Law']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 0)
#count_vect = CountVectorizer()
#X_train_counts = count_vect.fit_transform(X_train)
#tfidf_transformer = TfidfTransformer()
#X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
dtc_clf = tree.DecisionTreeClassifier()
dtc_clf = dtc_clf.fit(X_train,y_train)



In [9]:
print (dtc_clf.score(X_train,y_train))

0.9969230769230769


In [88]:
#Overfitted
print(dtc_clf.predict(count_vect.transform(["AT the outset I should like to point out that if this man was guilty of receiving a stolen cow, then a fine of Rs. 200/- was quite inadequate. Life in,villages in Bengal and elsewhere would become quite impossible if theft of cattle and the crime of receiving stolen cattle were dealt with leniently. Stealing a villager's cow is a very serious offence, and so is, being in possession of such a cow knowing it to be stolen. But for the view I take on the question of guilt, I should have no hesitation in issuing a notice to enhance this sentence. THE case for the prosecution was that the cow in question belonged to the complainant Harimati Dasi. The cow was found to be missing about the end of February 1948 and on March 12, 1948 the complainant reported the matter to the Ekbalpore Police. About two months after she had given the Police information, her missing cow, it was said, was found in the possession of one Fouzdar Sardar. The complainant then filed a petition of complaint against Fouzdar Sardar and the cow was seized under a search warrant issued by a learned Magistrate against Fouzdar Sardar. Whilst the case was pending, Fouzdar Sardar died and the person who was holding the cow was directed to make it over to the complainant. But he refused to do it and the cow was again seized under a search warrant.THE present petitioner Bharadwaj then filed a petition in the court of the Police Magistrate at Alipore claiming this cow as his own and stating that Fouzdar Sardar had been looking after the cow on his behalf. The petitioner claimed the cow and notice was issued upon the complainant who promptly retaliated by praying for process against Bharadwaj under   Section 411, I. P. C.EVIDENCE was adduced before the learned Magistrate that the complainant's cow had strayed and that the cow eventually found in possession of Fouzdar Sardar was the complainant's cow. This evidence was hotly contested by the defence. But I shall assume that it was proved satisfactorily that the cow in the possession of Fouzdar Sardar was the cow which has strayed from the complainant's premises about two or three months earlier.IT was argued in the lower courts that there was no evidence that this cow had been stolen. All the evidence amounts to is that it disappeared and of course it may have strayed. Taking possession however of a straying cow knowing it to be an animal which has strayed might well amount to theft, and I shall assume that this cow was stolen from the complainant by someone. BEING in possession of stolen property is in itself no offence. If it was, all of us I imagine would be in serious danger constantly. Any one in this Court might buy an article today quite openly and honestly and later find that that article had been stolen. If being in possession of stolen property was itself a crime, then an innocent purchase of that sort would render the purchaser liable to conviction for a serious offence. Being in possession of stolen property is only a crime if the person in possession either knows that the property is stolen property or has reasonable grounds for believing that the property had been stolen.TO succeed in a prosecution under  Section 411 I. P. C. , the prosecution must not only prove that the property had been stolen, but they must also, establish facts from which the Court can properly infer that the person charged with, being in possession of stolen property either knew the property to be stolen, or had reasonable grounds for believing the same to have been stolen. Unless there is some prima facie evidence as."])))

['Civil Procedure']


In [24]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

texts = df['clean_text'].astype('str')

tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2), 
                                   min_df = 2, 
                                   max_df = .95)

X = tfidf_vectorizer.fit_transform(texts) #features
y = df['Area.of.Law']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 0, shuffle=True)
#X_train, X_test, y_train, y_test = train_test_split(df['Content'], df['Area.of.Law'],test_size=0.3, random_state = 0)
#count_vect = CountVectorizer()
#X_train_counts = count_vect.fit_transform(X_train)
#tfidf_transformer = TfidfTransformer()
#X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
rfc_clf = RandomForestClassifier(max_depth=10, random_state=20, n_estimators=90)
rfc_clf.fit(X_train_tfidf,y_train)
print (rfc_clf.score(X_train_tfidf,y_train))
y_pred= rfc_clf.predict(X_train_tfidf)

#using a cv score RF might be overfitted
cv= cross_val_score(rfc_clf, X, y, cv=10)
print("Mean 5-Fold R Squared: {}".format(np.mean(cv)))

0.9323076923076923


C:\Users\Steven\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


Mean 5-Fold R Squared: 0.42307950120665305


In [21]:
print(rfc_clf.predict(count_vect.transform(["From time to time, the matter was adjourned as it could not be taken up on the grounds that the counsel were not available at the time of hearing. Finally, the matter was taken up</FC>.The claim of the petitioning creditor arises out of an export contract for supply of various qualities and quantities of A.C.S.R. conductors by the company to the foreign buyer in Pakistan being the Pakistan Water and Power Development Authority, hereinafter referred to as W.A.P.D.A. The said transaction for the supply to the foreign buyer by the company was routed through the petitioning creditor as the agent and also through the Industrial Import Private Limited, 34, Stephen House, 4, B.B.D. Bag (East), Calcutta 1, and Associated Commercial Agencies, Associate Chambers, 24/25, Patiala Ground Link, Mcleod Road, Lahore, who were acting as the Indian and Pakistani agent of the petitioning creditor. The said W.A.P.D.A. of Pakistan invited a global tender due on December 20, 1976, for the purchase of the said goods. Thereafter, the petitioning creditors said agent in India invited offers from Indian Manufacturers of the said articles accordingly and the company submitted its offer together with quotation by its letter dated December 11, 1976, which is set out hereunder:"])))

['Property Laws']


In [153]:
#1988 114
print(rfc_clf.predict(count_vect.transform(["By a registered Deed of lease dated 16th of September 1959, the petitioner company who was the owner of' the Jute Mill in Shibpur, Howrah granted a lease of a portion of its Mill promises in favour of the Respondent No. 5, Remington Rand of India Ltd. for a term of 20 years begining from let of September 1959 which expired with effect from 1st of September 1979 at a monthly rent of Rs. 17,250/ ?. Before the expiry of the said lease period of 20 years the petitioner company by the notice dated 7th of February 1979 required the respondent no. 5, Remington Rand of India Ltd. to deliver peaceful and vacant possession of the said premises to the petitioner company on the expiry of the period stipulated in the Deed of Lease dated 16th September, 1959. After the lease period-expired with effect from 1st September 1979, the Respondent company did not deliver possession of the said premises. Thereafter, on 21st of April, 1980 the petitioner company filed a suit for eviction of the respondent company in Title Suit No. 46 of 1980 in the Court of Subordinate Judge, Howrah. Upon the petitioners' application made before this High Court under Clause 13 of the Letters Patent for transfer of the said suit to this Hon'ble court, the said suit was transferred before this Court for expeditious hearing on the basis of the allegation that the Respondents were delaying the final hearing of the said suit on some plea or other. On 29th May, 1985, the said suit appeared for final hearing and on that day the Respondent Company moved an application for amendment of the written statement. The said amendment was allowed by His Lordship Hon'ble Mr. Justice B. C. Basak of this Court and the matter was directed to be listed on 4th june, 1985."])))
#1956 163
print(rfc_clf.predict(count_vect.transform(["This is an appeal from an order of Sinha, J., dated 27-7-1955, whereby the learned Judge quashed an order of assessment in respect of a property owned by the respondent and gave certain consequential directions. The appellant is the Corporation of Calcutta. The premises concerned is situated in an area which appertained formerly to what was once the Tollygunge Municipality. It appears that in the year 1951, there was a general assessment and in the course of that assessment, the respondent's property which is Premises No. 46 Izazatulla Lane was assessed to a valuation of Rs. 270/-. The Tollygunge Municipality was superseded by an order of the State of West Bengal on 26-11-1951, for the period of one year, but the period of supersession was subsequently extended from time to time. Upon superseding the Municipality, the State of West Bengal appointed one Shri Priya Nath Bose to be the Administrator and to exercise all the powers and discharge all the duties which might be done by the Chairman and the Commissioners of the Municipality, whether at a meeting or otherwise. The Administrator, upon assuming office, made special arrangements for a public exhibition of the new assessment lists and those were exhibited on 2-1-1952. The respondent who was dissatisfied with the assessment made upon his property filed an objection before the Administrator under Section 148 Bengal Municipal Act, That objection was filed on 30-1-1952 and was disposed of on 21-5-1952. The Administrator gave effect to the respondent's objection to a certain extent and reduced the valuation from Rs. 270/- to Rs. 240/-."])))
#1976 277
print(rfc_clf.predict(count_vect.transform(["THE plaintiff opposite party, Sadhan Chandra Barik filed a title suit, being t. S. No. 93 of 1974, in the First Court of the Munsif atDiamondHarbour against the defendant-petitioner praying for a declaration of title and for injunction in respect of the suit laid described in schedule Khas to the plaint. The defendant-petitioner entered appearance and claimed that he was a Bargadar in respect of the suit land and on april 7, 1975 filed an application before the learned Munsif for referring to the bhagchas Officer for a decision of the question whether the defendant was a bargadar or not in accordance with the provisions of  section 21 (3) of the West. Bengal Land Reforms Act , 1955 (hereinafter referred to as 'the Act). The learned Munsif by order No. 26 dated june 17, 1975 rejected the petitioner's application holding, inter alia, that the dispute between the parties was the subject matter of a suit'and the order prayed for would tantamount to the passing of an interlocutory order in the suit as it would not be a proceeding and as such as in    section 21 (3) of the act the word proceeding and not 'suit' has been used he had no authority to refer the question to the Bhagchas officer. This Order of the learned munsif has been impugned in this Rule. IN my view, this application must succeed Section 21 (3) of the Act is in the following terms :- if any question as to whether a person is or is rot a bargadar arises in the course of any proceedings before any Civil or Criminal Court, the court shall refer to the officer or authority mentioned in sub-section (1) of   section 18 lord decision. IT is clear from a reading of the aforesaid sub-section that the word proceeding is wide enough to cover a suit. It this was not so, then, if a question whether a person was or was not a bargadar arose in a suit, then the authority constituted under the Act to determine this question would have no jurisdiction to do. This could not possibly be the intention of the legislature which has created a special authority under the Act to determine the aforesaid question.THIS <DP>application accordingly succeeds </DP>and the Rule is made absolute. Order No. 26 dated 17. 6. 75, passed by the learned Munsif, First Court, diamond Harbour in Title Suit No. 93/74 is set aside. The learned Munsif will refer to the appropriate authority under the Act the question whether the defendant is a bargadar or not and after getting a report of the competent authority, he will proceed with the suit in accordance with law. </HD>There will be no orders as to costs in this Rule. Rule made absoluteIN the instant case in the course of wearing of the suit a question was raised whether the defendant was a bargadar or not. The petition by the defendant to the court for a reference to the Bhagchas Officer is therefore competent and the learned Munsif should have referred the matter accordingly under section 21 (3) of the Act to the authority contemplated under the Act."])))

['Tenancy Laws']
['Tenancy Laws']
['Civil Procedure']


In [147]:
#1980 52
print(rfc_clf.predict(count_vect.transform(["And the deponent due to the insertion of  Section 16A of the MISA and curtailment of the powers of court was not aware of the grounds of detention of her husband nor she could invoke the writ jurisdiction of the courts. The copy of the second detention order has been annexed to the affidavit. After the declaration of the Loksabha poll the husband of the deponent was released, and thereafter the affidavit had been made. The two detention orders, first one dated 19.7.75 and the second one dated 27.9.75. are as follows No. MISA. 231/75 Dated-19.7.75 Whereas I am satisfied with respect to the person known as Shri Sashi Kanta Chakraborty, son of Late Jiminy Kanta Chakraborty of Singa Tola, P.S. Englishbazar and/of Chanchal, P.S. Kharba, District Malda, that with a view to prevent him from acting in any manner prejudicial to the maintenance of public order, it is necessary so to do. Now, therefore, in exercise of the powers conferred by Sub-   section (1) read with Sub-section (2) of Section 3 of the Maintenance of Internal Security Act , 1971 (Act 26 of 1971) I hereby make this order directing the said Shri Sashi Kanta Chakraborty be detained. No. MISA. 285/75 Dated 27.9.75 Whereas I am satisfied with respect to the person known as Shri Sashi Kanta Chakraborty, son of late Jiminy Kanta Chakraborty of Singatola, P. S. English bazaar and of Chanchal, P. S. Chanchal, District Malda, that with a view to prevent him from acting in any manner pre-judicial to the maintenance of public order security of the State, its necessary so to do Now, therefore in exercise of the powers conferred by Sub    section (1) read with Sub-section (2) of Section 3 of the Maintenance of Internal Security Act, 1971 (Act 26 of 1971) I hereby make this order directing the said Shri Sashi Kanta Chakraborty be detained.The deponent has further alleged that due to such wrongful confinement the deponent and her family had been in financial stress and were reduced to distress. In the premises, the dep "])))

['Civil Procedure']
